In [1]:
from dotenv import load_dotenv; load_dotenv()

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
from tqdm import tqdm

from pandarallel import pandarallel
pandarallel.initialize()
from mlb_dataloader.util.data_helpers import sql_read

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# readers

In [2]:
readers = sql_read(f'''SELECT * FROM "HAHATON".data_reader''')
print(readers.shape)
readers.head()

(562133, 3)


,id,birth_date,age
0,366670,1974-10-14,46.079452
1,235264,1978-06-01,42.446575
2,133225,1976-01-06,44.849315
3,556716,1989-01-28,31.778082
4,52086,1955-08-14,65.260274


# catalog

In [47]:
catalog = sql_read(f'''SELECT * FROM "HAHATON".data_catalog''')
print(catalog.shape)
catalog.head()

(1370457, 11)


,id,author,title,city,publisher,year,series,tags,article,age_rating,cover_url
0,1,Устинова Татьяна Витальевна,Призрак Канта,Москва,Эксмо,2018,Татьяна Устинова. Первая среди лучших,Художественная литература ; Российский детектив,84(2Рос)6,16+,None
1,2,None,Наука и жизнь,Москва,АНО Редакция журнала «Наука и жизнь»,1938-,nan,Наука. Науковедение,72,12+,None
2,11185,None,Путешествие на зеленый свет (или Школа юного п...,Москва,Кедр,None,nan,Техника и технические науки,3,6+,None
3,11157,None,Мурзилка,Москва,"Редакция журнала ""Мурзилка""",1924-,nan,Литература для детей и юношества,84,6+,None
4,11161,None,Нарконет,Москва,Нарконет,1999-,nan,Здравоохранение и медицина,5,12+,None


In [48]:
catalog[catalog.id == 594528]

,id,author,title,city,publisher,year,series,tags,article,age_rating,cover_url
817130,594528,Машинский Семен Иосифович,С. Т. Аксаков. Жизнь и творчество,Москва,Гослитиздат,1961,nan,nan,8Р1XIX,None,None


### Проблема: у многих записей в каталоге одинаковые автор и заглавие. 
### Необходимо чтобы это считалось одним экземпляром

In [4]:
catalog['unique'] = catalog[['author', 'title']].parallel_apply(
    lambda x: frozenset(str(x[0]).strip().split()) | frozenset(str(x[1]).strip().split()), axis=1)
catalog.head()

,id,author,title,city,publisher,year,series,tags,article,age_rating,cover_url,unique
0,1,Устинова Татьяна Витальевна,Призрак Канта,Москва,Эксмо,2018,Татьяна Устинова. Первая среди лучших,Художественная литература ; Российский детектив,84(2Рос)6,16+,None,"(Татьяна, Устинова, Витальевна, Канта, Призрак)"
1,2,None,Наука и жизнь,Москва,АНО Редакция журнала «Наука и жизнь»,1938-,nan,Наука. Науковедение,72,12+,None,"(None, и, жизнь, Наука)"
2,11185,None,Путешествие на зеленый свет (или Школа юного п...,Москва,Кедр,None,nan,Техника и технические науки,3,6+,None,"(зеленый, свет, на, Школа, (или, юного, пешехо..."
3,11157,None,Мурзилка,Москва,"Редакция журнала ""Мурзилка""",1924-,nan,Литература для детей и юношества,84,6+,None,"(Мурзилка, None)"
4,11161,None,Нарконет,Москва,Нарконет,1999-,nan,Здравоохранение и медицина,5,12+,None,"(None, Нарконет)"


In [5]:
t = catalog[['unique', 'id']].groupby('unique')['id'].agg(list)


catalogid_to_uniquecatalogid = dict()

for ids in tqdm(t.values):
    for id_ in ids:
        catalogid_to_uniquecatalogid[id_] = ids[0]
        
len(catalogid_to_uniquecatalogid)

# В дальнейшем наложим эту маску на таблицу экземпляров

100%|██████████| 690608/690608 [00:00<00:00, 2143837.19it/s]


1370457

# book_copy

In [6]:
book_copy = sql_read(f'''SELECT * FROM "HAHATON".data_bookcopy''')
print(book_copy.shape)
book_copy.head()

(12480044, 6)


,id,inventory_id,barcode,classification_id,sigles_id,book_id
0,12488544,09:0000120386,980007854120,84(2Рос=Рус)6,314.0,1
1,12488546,09:0000152427,980010247056,84(2Рос=Рус)6,320.0,1
2,12488542,09:0000240622,980000737154,84(2Рос=Рус)6,306.0,1
3,13671389,09:0000278133,980005475303,84(2Рос=Рус)6,308.0,1
4,7625486,05:0000257728,580001931461,84(2Рос=Рус)6,293.0,1


In [7]:
book_copy['book_id'] = book_copy['book_id'].map(catalogid_to_uniquecatalogid)

# booktake

In [8]:
booktake = sql_read(f'''SELECT * FROM "HAHATON".data_booktake''')
print(booktake.shape)
booktake.head()

(10411858, 7)


,id,inventory_id,barcode,take_at,return_at,condition_id,reader_id
0,4509252,Б06:00004188,680001558184,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331500
1,4509253,Б06:00004195,680001558139,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331500
2,4509254,07:0000120164,780002213645,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331532
3,4509255,07:0000120151,780002213638,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331532
4,4509256,07:0000120139,780002213621,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331532


# Processing...

In [9]:
readers = readers.merge(booktake['reader_id'].value_counts().reset_index().rename(
    columns={'index': 'id',
             'reader_id': 'book_count'}),
              on='id', how='left')
print(readers.shape)
readers.head()

(562133, 4)


,id,birth_date,age,book_count
0,366670,1974-10-14,46.079452,5.0
1,235264,1978-06-01,42.446575,NaN
2,133225,1976-01-06,44.849315,NaN
3,556716,1989-01-28,31.778082,NaN
4,52086,1955-08-14,65.260274,NaN


In [10]:
readers.fillna(0, inplace=True)

In [13]:
len(set(booktake['inventory_id']) - set(book_copy['inventory_id']))

67799

In [14]:
booktake = booktake.merge(book_copy[['inventory_id', 'book_id']], on='inventory_id', how='left')
print(booktake.shape)
booktake.head()

(10411858, 8)


,id,inventory_id,barcode,take_at,return_at,condition_id,reader_id,book_id
0,4509252,Б06:00004188,680001558184,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331500,11169.0
1,4509253,Б06:00004195,680001558139,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331500,11169.0
2,4509254,07:0000120164,780002213645,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331532,163585.0
3,4509255,07:0000120151,780002213638,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331532,1250945.0
4,4509256,07:0000120139,780002213621,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331532,1256585.0


In [15]:
booktake['book_id'].isna().mean()

0.022243100126797734

In [16]:
booktake = booktake.merge(catalog[['id', 'author', 'title', 'year', 'series', 
                                   'tags', 'age_rating']].rename(columns={'id': 'book_id'}), 
                on='book_id', how='left')
print(booktake.shape)
booktake.head()

(10411858, 14)


,id,inventory_id,barcode,take_at,return_at,condition_id,reader_id,book_id,author,title,year,series,tags,age_rating
0,4509252,Б06:00004188,680001558184,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331500,11169.0,None,Новый мир,1925-,nan,Художественная литература,16+
1,4509253,Б06:00004195,680001558139,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331500,11169.0,None,Новый мир,1925-,nan,Художественная литература,16+
2,4509254,07:0000120164,780002213645,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331532,163585.0,Абитан Анн-Мари,Как мышонок учился читать,2014,nan,Литература для детей и юношества,0+
3,4509255,07:0000120151,780002213638,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331532,1250945.0,None,Медвежонок Винни и его друзья. День чистоты,2016,Disney,Литература для детей и юношества,0+
4,4509256,07:0000120139,780002213621,2019-11-26 15:00:00+00:00,2019-11-26 15:00:00+00:00,6548,331532,1256585.0,Казалис Анна,Мышонок Тим капризничает,2019,Мышонок Тим,Литература для детей и юношества,0+


In [19]:
booktake = booktake[~booktake['book_id'].isna()].reset_index(drop=True)
print(booktake.shape)

(10180266, 14)


In [21]:
booktake = booktake.drop_duplicates(['reader_id', 'book_id', 'take_at']).reset_index(drop=True)
print(booktake.shape)

(7697843, 14)


In [29]:
# популярность книги

t = booktake.groupby('book_id')['year'].agg(['count']).reset_index()
t

,book_id,count
0,1.0,1884
1,2.0,26459
2,6.0,17
3,9.0,3
4,10.0,2
...,...,...
320129,1783590.0,1
320130,1783591.0,2
320131,1783601.0,47
320132,1783602.0,46


In [38]:
catalog = catalog.merge(t.rename(columns={'book_id': 'id'}), on='id', how='left')
catalog['count'] = catalog['count'].fillna(0)
catalog.head()

,id,author,title,city,publisher,year,series,tags,article,age_rating,cover_url,unique,count
0,1,Устинова Татьяна Витальевна,Призрак Канта,Москва,Эксмо,2018,Татьяна Устинова. Первая среди лучших,Художественная литература ; Российский детектив,84(2Рос)6,16+,None,"(Татьяна, Устинова, Витальевна, Канта, Призрак)",1884.0
1,2,None,Наука и жизнь,Москва,АНО Редакция журнала «Наука и жизнь»,1938-,nan,Наука. Науковедение,72,12+,None,"(None, и, жизнь, Наука)",26459.0
2,11185,None,Путешествие на зеленый свет (или Школа юного п...,Москва,Кедр,None,nan,Техника и технические науки,3,6+,None,"(зеленый, свет, на, Школа, (или, юного, пешехо...",0.0
3,11157,None,Мурзилка,Москва,"Редакция журнала ""Мурзилка""",1924-,nan,Литература для детей и юношества,84,6+,None,"(Мурзилка, None)",13905.0
4,11161,None,Нарконет,Москва,Нарконет,1999-,nan,Здравоохранение и медицина,5,12+,None,"(None, Нарконет)",11.0


In [39]:
catalog['rating'] = np.log(catalog['count'].values + 1)
catalog.head()

,id,author,title,city,publisher,year,series,tags,article,age_rating,cover_url,unique,count,rating
0,1,Устинова Татьяна Витальевна,Призрак Канта,Москва,Эксмо,2018,Татьяна Устинова. Первая среди лучших,Художественная литература ; Российский детектив,84(2Рос)6,16+,None,"(Татьяна, Устинова, Витальевна, Канта, Призрак)",1884.0,7.541683
1,2,None,Наука и жизнь,Москва,АНО Редакция журнала «Наука и жизнь»,1938-,nan,Наука. Науковедение,72,12+,None,"(None, и, жизнь, Наука)",26459.0,10.183389
2,11185,None,Путешествие на зеленый свет (или Школа юного п...,Москва,Кедр,None,nan,Техника и технические науки,3,6+,None,"(зеленый, свет, на, Школа, (или, юного, пешехо...",0.0,0.000000
3,11157,None,Мурзилка,Москва,"Редакция журнала ""Мурзилка""",1924-,nan,Литература для детей и юношества,84,6+,None,"(Мурзилка, None)",13905.0,9.540076
4,11161,None,Нарконет,Москва,Нарконет,1999-,nan,Здравоохранение и медицина,5,12+,None,"(None, Нарконет)",11.0,2.484907


In [45]:
ratings = booktake[['reader_id', 'book_id']].merge(
    catalog[['id', 'rating']].rename(columns={'id': 'book_id'}), 
    on='book_id', how='left').fillna(0)
print(ratings.shape)
ratings

(7697843, 3)


,reader_id,book_id,rating
0,331500,11169.0,9.065892
1,331532,163585.0,6.800170
2,331532,1250945.0,4.276666
3,331532,1256585.0,4.919981
4,26581,909409.0,5.981414
...,...,...,...
7697838,127636,155292.0,3.891820
7697839,322487,222308.0,5.159055
7697840,322487,452639.0,6.827629
7697841,273815,299759.0,3.951244


In [46]:
ratings.to_csv('ratings.csv', index=False)

In [44]:
catalog.to_csv('books.csv', index=False)

In [53]:
ratings_test = sql_read(f'''SELECT * FROM "HAHATON".data_mlrating''')

In [49]:
catalog_test = sql_read(f'''SELECT * FROM "HAHATON".data_catalog''')

In [55]:
set(ratings_test['doc_id']) - set(catalog_test['id'])

set()

In [52]:
set(ratings['book_id']) - set(catalog_test['id'])

set()

In [56]:
readers

,id,birth_date,age,book_count
0,366670,1974-10-14,46.079452,5.0
1,235264,1978-06-01,42.446575,0.0
2,133225,1976-01-06,44.849315,0.0
3,556716,1989-01-28,31.778082,0.0
4,52086,1955-08-14,65.260274,0.0
...,...,...,...,...
562128,554754,1981-03-08,39.676712,0.0
562129,405813,2009-07-07,11.326027,6.0
562130,534901,2005-07-28,15.271233,4.0
562131,50142,2011-11-12,8.975342,51.0


In [74]:
events = sql_read(f'''SELECT * FROM "HAHATON".data_event''')
print(events.shape)
events.head()

(41072, 28)


,id,name,status,host_name,host_type,host_subclass,price,event_type,event_focus,is_holiday,...,district,area,is_disabled_friendly,disability_type,age_limit,age_group,target_audience,age_group_floor,age_group_ceil,mosru_id
0,1,Врасплох,Завершено,ГБУК г. Москвы «ДК «Нагатино»,Дома культуры,nan,бесплатное,выставка,антитеррор,nan,...,Южный административный округ,Нагатинский Затон,nan,nan,6,от 16 до 100,"школьники, работающая молодежь, студенческая м...",16,100,3179
1,2,Праздник Новогодней елки. Новогодний концерт в...,Завершено,ГАУК г. Москвы «Культурный центр ЗИЛ»,Дома культуры,nan,бесплатное,"концерт, музыкальное представление",культурно-досуговое,праздник,...,nan,nan,nan,nan,0,от 5 до 14,"дети дошкольного возраста, школьники",5,14,486
2,3,"Выставка к Рождеству ""Сувенир Года""",Завершено,ГБУК г.Москвы ТКС «Орехово»,Дома культуры,nan,бесплатное,выставка,культурно-досуговое,праздник,...,Южный административный округ,Орехово-Борисово Северное,недоступно,nan,6,от 15 до 80,"дети дошкольного возраста, школьники, работающ...",15,80,2100
3,4,Зимняя сказка,Завершено,ГБУК г. Москвы «КЦ «Зодчие»,Дома культуры,nan,бесплатное,"презентация, демонстрация, показетльные выступ...",культурно-досуговое,нет,...,Западный административный округ,Кунцево,недоступно,nan,0,от 5 до 80,"дети дошкольного возраста, школьники",5,80,254
4,5,Почта Деда Мороза,Завершено,"ГБУК г. Москвы ""ДК ""Темп""",Дома культуры,nan,бесплатное,акция,сохранение семейных ценностей,праздник,...,nan,nan,недоступно,nan,0,от 6 до 99,nan,6,99,78


In [58]:
age = 8

In [62]:
from datetime import date
date.today()

datetime.date(2020, 11, 1)

In [104]:
age = 55

np.random.seed(age)


In [105]:
events = sql_read(f'''SELECT * FROM "HAHATON".data_event''')
events = events[events.start_date > date.today()]

if age < 12:
    ids = np.random.choice(events[(events.age_group_ceil >= 0) & (events.age_group_ceil < 18)]['id'], 10, replace=False)
elif age < 18:
    ids = np.random.choice(events[(events.age_group_ceil >= 0) & (events.age_group_ceil < 40)]['id'], 10, replace=False)
elif age < 50:
    ids = np.random.choice(events[(events.age_group_ceil >= 18) & (events.age_group_ceil < 999)]['id'], 10, replace=False)
else:
    ids = np.random.choice(events[(events.age_group_ceil >= 55) & (events.age_group_ceil < 999)]['id'], 10, replace=False)
    
events[events.id.isin(ids)]

,id,name,status,host_name,host_type,host_subclass,price,event_type,event_focus,is_holiday,...,district,area,is_disabled_friendly,disability_type,age_limit,age_group,target_audience,age_group_floor,age_group_ceil,mosru_id
32702,32688,"Спектакль театра-студии ""Традиция""",Уточняется,ГБУК г. Москвы «КЦ «Москвич»,Дома культуры,nan,бесплатное,"представление, спектакль, инсценировка, перфом...",культурно-досуговое,нет,...,Юго-Восточный административный округ,Текстильщики,недоступно,nan,6,от 0 до 99,"школьники, работающая молодежь, студенческая м...",0,99,1511
33813,33778,Королева законов,Запланировано,ГБУК г.Москвы «ЦБС ЮВАО»,Библиотеки,nan,бесплатное,лекция,просветительское,nan,...,Юго-Восточный административный округ,Некрасовка,доступно,Опорно-двигательный аппарат,12,от 0 до 99,"школьники, работающая молодежь, студенческая м...",0,99,1577
34800,34757,Новогодняя сказка,Уточняется,ГБУК г. Москвы «ТКС «Товарищ»,Дома культуры,nan,бесплатное,"представление, спектакль, инсценировка, перфом...",культурно-досуговое,событие,...,Северо-Восточный административный округ,Ростокино,недоступно,nan,0,от 12 до 80,"дети дошкольного возраста, школьники, работающ...",12,80,1443
36397,37227,Танцевальная практика,Запланировано,ГБУК г.Москвы «ДК «Нагорный»,Дома культуры,nan,платное (от 150 руб. до 150 руб.),"танцевальный вечер, бал",культурно-досуговое,nan,...,Южный административный округ,Нагорный,nan,nan,18,от 30 до 70,nan,30,70,145
37214,37189,Танец - повод для знакомства,Запланировано,ГБУК г.Москвы «ДК «Нагорный»,Дома культуры,nan,платное (от 150 руб. до 150 руб.),"игра, игровая программа",культурно-досуговое,нет,...,Южный административный округ,Нагорный,недоступно,nan,18,от 50 до 95,взрослые,50,95,92
37223,37197,Танцевальная практика,Запланировано,ГБУК г.Москвы «ДК «Нагорный»,Дома культуры,nan,платное (от 150 руб. до 150 руб.),"танцевальный вечер, бал",культурно-досуговое,nan,...,Южный административный округ,Нагорный,nan,nan,18,от 50 до 90,nan,50,90,102
37242,37226,Танцевальная практика,Запланировано,ГБУК г.Москвы «ДК «Нагорный»,Дома культуры,nan,платное (от 150 руб. до 150 руб.),"танцевальный вечер, бал",культурно-досуговое,nan,...,Южный административный округ,Нагорный,nan,nan,18,от 30 до 70,nan,30,70,143
37304,37293,Фотоархив,Утверждено,ГБУК г. Москвы «ЦБС ЦАО»,Библиотеки,nan,бесплатное,"встреча, творческий вечер",культурно-досуговое,нет,...,Центральный административный округ,Басманный,доступно,Опорно-двигательный аппарат,18,от 18 до 75,"работающая молодежь, студенческая молодежь",18,75,481
37373,37360,"Эдуард Успенский: личность, жизнь, герои",Запланировано,"ГБУК г. Москвы ""КЦ""Акулово""",Дома культуры,nan,бесплатное,"встреча, творческий вечер",культурно-досуговое,nan,...,Восточный административный округ,Восточный,доступно,Опорно-двигательный аппарат,6,от 6 до 100,"школьники, работающая молодежь, студенческая м...",6,100,964
37384,37370,Территория безопасности,Уточняется,ГБУК г. Москвы «ДК «Содружество»,Дома культуры,nan,бесплатное,лекция,антитеррор,нет,...,Северо-Восточный административный округ,Алексеевский,недоступно,nan,6,от 6 до 80,"дети дошкольного возраста, школьники, работающ...",6,80,1066
